In [26]:
#Импортируем нужные библиотеки
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PowerTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump, load

In [27]:
#Считываем данные
df = sns.load_dataset('diamonds').drop(columns=["color", "clarity", "depth", "table"])
df = df.sample(df.shape[0], ignore_index=True)
df.sample(10)

,carat,cut,price,x,y,z
28538,0.43,Premium,968,4.87,4.84,2.96
53204,0.90,Very Good,3342,6.07,6.02,3.83
23420,0.40,Premium,945,4.71,4.67,2.93
20302,0.51,Ideal,1343,5.09,5.14,3.19
6883,1.01,Very Good,3852,6.43,6.46,3.99
45505,0.51,Very Good,2146,5.06,5.10,3.19
50544,1.31,Ideal,12825,6.97,7.00,4.34
7612,0.91,Very Good,3884,6.12,6.17,3.82
10105,0.55,Ideal,1871,5.31,5.29,3.27
38992,0.69,Fair,2235,5.58,5.64,3.42


In [28]:
#Смотрим информацию:тип данных и количество пропусков
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   carat   53940 non-null  float64 
 1   cut     53940 non-null  category
 2   price   53940 non-null  int64   
 3   x       53940 non-null  float64 
 4   y       53940 non-null  float64 
 5   z       53940 non-null  float64 
dtypes: category(1), float64(4), int64(1)
memory usage: 2.1 MB


In [29]:
#Смотрим количество пропусков в датасете
df.isna().sum()

carat    0
cut      0
price    0
x        0
y        0
z        0
dtype: int64

In [30]:
#Наблюдаем количество дубликатов
df.duplicated().sum()

457

In [31]:
#Удаляем дубликаты
df = df.drop_duplicates()
df.shape

(53483, 6)

In [32]:
#посмотрим на то, какие у нас есть категориальные и численные столбцы

cat_columns = [] # создаем пустой список для имен колонок категориальных данных
num_columns = [] # создаем пустой список для имен колонок числовых данных

for column_name in df.columns: # смотрим на все колонки в датафрейме
    if (df[column_name].dtypes == 'category'): # проверяем тип данных для каждой колонки
        cat_columns +=[column_name] # если тип объект - то складываем в категориальные данные
    else:
        num_columns +=[column_name] # иначе - числовые

# важно: если признак категориальный, но хранится в формате числовых данных, тогда код не сработает корректно


# выводим результат
print('Категориальные данные:\t ',cat_columns, '\n Число столблцов = ',len(cat_columns))

print('Числовые данные:\t ',  num_columns, '\n Число столблцов = ',len(num_columns))

Категориальные данные:	  ['cut'] 
 Число столблцов =  1
Числовые данные:	  ['carat', 'price', 'x', 'y', 'z'] 
 Число столблцов =  5


### Создание датасетов

In [33]:
!mkdir df_1 df_2 df_3 df_4

In [34]:
#Создание датасетов (Деление на равные датасеты)
def df_create(data, n):
    
    num_dir = 0
    before_index = 0
    shape = data.shape[0]
    
    for index in range(int(shape / n), shape, int(shape / n)):
        num_dir += 1
        create_df = data.iloc[before_index:index]
        
        #Создаем n-й датасет с шумом
        if num_dir == 4:
            create_df["price"] = create_df["price"] + \
                100 * np.random.randn(create_df.shape[0])

        create_df.to_csv(f"df_{num_dir}/df.csv", index=False)
            
        before_index = index

        

In [35]:
df_create(df, 4)

C:\Users\v.zemov\AppData\Local\Temp\ipykernel_11252\30957057.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_df["price"] = create_df["price"] + \


In [36]:
df_1 = pd.read_csv('df_1/df.csv')
df_2 = pd.read_csv('df_2/df.csv')
df_3 = pd.read_csv('df_3/df.csv')
df_4 = pd.read_csv('df_4/df.csv')

### Препроцессинг и обучение модели

In [37]:
# Создание пайплайна для препроцессинга данных и выполнения прогноза

pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            (
                "encoder",
                OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse=False),
                ['cut'],
            ),
            ("scaler", StandardScaler(), ['carat', 'x', 'y', 'z']),
        ]
    ),
    LinearRegression(),
)

In [38]:
X = df_1.drop(columns=["price"])
y = df_1["price"]

# Разделение на обучающий и тестовый наборы данных

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Размер  (X_train):", X_train.shape)
print("Размер (X_test):", X_test.shape)
print("Размер  (y_train):", y_train.shape)
print("Размер (y_test):", y_test.shape)

Размер  (X_train): (10696, 5)
Размер (X_test): (2674, 5)
Размер  (y_train): (10696,)
Размер (y_test): (2674,)


In [39]:
#Обучение модели
model_predict = pipeline.fit(X_train, y_train)

In [40]:
# Сохранение обученной модели
dump(model_predict, "model_pedict_price.joblib")

['model_pedict_price.joblib']

In [41]:
# Прогнозирование на тестовом наборе данных
y_pred = model_predict.predict(X_test)

In [42]:
# Вычисление средней абсолютной ошибки (MAE) и коэффициента детерминации(R^2)
MAE = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Средняя абсолютная ошибка (MAE):", MAE)
print("Коэффициент детерминации (R^2):", r2)

Средняя абсолютная ошибка (MAE): 904.5713818249812
Коэффициент детерминации (R^2): 0.8648892632795175


### Создаем файл для теста

In [74]:
%%writefile test.py
import numpy as np
import pandas as pd
import pytest
from joblib import load
from sklearn.metrics import mean_absolute_error, r2_score


@pytest.fixture()
def load_model():
    model = load("model_pedict_price.joblib")
    return model


@pytest.fixture()
def making_prediction(load_model):
    data = pd.read_csv("df.сsv")
    X = data.drop(columns="price")
    y = data["price"]
    y_pred = load_model.predict(X)
    return y, y_pred


def test_MAE(making_prediction):
    y, y_pred = making_prediction
    assert mean_absolute_error(y, y_pred) < 900


def test_R2(making_prediction):
    y, y_pred = making_prediction
    assert r2_score(y, y_pred) > 0.85
    

Writing test.py


### Тестируем датафреймы

In [80]:

!pytest -v test.py
#%cd ..

============================= test session starts =============================
platform win32 -- Python 3.9.12, pytest-7.1.1, pluggy-1.0.0 -- C:\Users\v.zemov\Anaconda3\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\v.zemov\Desktop\Master\2 семестр ИИ\Автоматизация машин_обучения\lab_5
plugins: anyio-3.7.1
collecting ... collected 2 items

test.py::test_MAE ERROR                                                  [ 50%]
test.py::test_R2 ERROR                                                   [100%]

=================================== ERRORS ====================================
_________________________ ERROR at setup of test_MAE __________________________

load_model = Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('encoder',
                   ...                               ['carat', 'x', 'y', 'z'])])),
                ('linearregression', LinearRegression())])

    @pytest.fixture()
    def making_prediction(load_model):
>       data 

__________________________ ERROR at setup of test_R2 __________________________

load_model = Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('encoder',
                   ...                               ['carat', 'x', 'y', 'z'])])),
                ('linearregression', LinearRegression())])

    @pytest.fixture()
    def making_prediction(load_model):
>       data = pd.read_csv("df.сsv")

test.py:16: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
..\..\..\..\..\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: in wrapper
    return func(*args, **kwargs)
..\..\..\..\..\Anaconda3\lib\site-packages\pandas\io\parsers\readers.py:680: in read_csv
    return _read(filepath_or_buffer, kwds)
..\..\..\..\..\Anaconda3\lib\site-packages\pandas\io\parsers\readers.py:575: in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
..\..\..\..\..\Anaconda3\lib\site-packages\pandas\io\parsers\readers.py:933: in

In [78]:
%pwd


'C:\\Users\\v.zemov\\Desktop\\Master\\2 семестр ИИ\\Автоматизация машин_обучения\\lab_5\\df_1'